In [2]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping, ReduceLROnPlateau

%matplotlib inline
import matplotlib.pyplot as plt

img_width, img_height = 224,224
train_data_dir = "train"
validation_data_dir = "val"
batch_size = 16
epochs = 50

# model = applications.densenet.DenseNet121(include_top=True, weights='imagenet', classes=1000)
model = applications.densenet.DenseNet121(include_top=True, weights='imagenet', input_shape = (img_width, img_height, 3), classes=1000)
model.layers.pop()

for layer in model.layers:
    layer.trainable = False

#Adding custom Layers 
x = model.layers[-1].output
x = Dropout(0.5)(x)
x = Dense(10, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation="sigmoid")(x)

# creating the final model 
model_final = Model(inputs = model.input, outputs = predictions)

# compile the model 
model_final.compile(loss = "binary_crossentropy", optimizer = optimizers.Adam(lr=0.001), metrics=["accuracy"])
model_final.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_3 (ZeroPadding2D (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_3[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [18]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Input, Conv2D, MaxPooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

# %matplotlib inline
# import matplotlib.pyplot as plt

train_data_dir = "train"
validation_data_dir = "val"
batch_size = 32
epochs = 50

img_width, img_height = 224, 224

### Build the network 
img_input = Input(shape=(img_width, img_height, 3))
x = Conv2D(24, (5,5), strides=(2,2), activation='relu')(img_input)
x = Conv2D(32, (5,5), strides=(2,2), activation='relu')(x)
x = Conv2D(64, (5,5), strides=(2,2), activation='relu')(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)
x = Conv2D(64, (3,3), strides=(1,1), activation='relu')(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)
x = Conv2D(128, (3,3), strides=(1,1), activation='relu')(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(100, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation="softmax")(x)

# creating the final model 
model_final = Model(inputs = img_input, outputs = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])
model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 110, 110, 24)      1824      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 53, 53, 32)        19232     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 25, 25, 64)        51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 10, 10, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
__________

In [ ]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Input, Conv2D, MaxPooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

# %matplotlib inline
# import matplotlib.pyplot as plt

train_data_dir = "train"
validation_data_dir = "val"
batch_size = 32
epochs = 50

img_width, img_height = 224, 224

### Build the network 
img_input = Input(shape=(img_width, img_height, 3))
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

# Block 2
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

model = Model(input = img_input, output = x)

layer_dict = dict([(layer.name, layer) for layer in model.layers if not layer.name.startswith('input')])
# model = applications.VGG19(include_top=True, weights='imagenet', classes=1000)
# model.save('vgg19_weights.h5')

import h5py
weights_path = 'vgg19_weights.h5' # ('https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels.h5)
f = h5py.File(weights_path)

# list all the layer names which are in the model.
layer_names = [layer.name for layer in model.layers]

"""
# Here we are extracting model_weights for each and every layer from the .h5 file

>>> f["model_weights"]["block1_conv1"].attrs["weight_names"]
array([b'block1_conv1/kernel:0', b'block1_conv1/bias:0'], 
      dtype='|S21')
# we are assiging this array to weight_names below 

>>> f["model_weights"]["block1_conv1"]["block1_conv1/kernel:0]
<HDF5 dataset "kernel:0": shape (3, 3, 3, 64), type "<f4">
# The list comprehension (weights) stores these two weights and bias of both the layers 

>>>layer_names.index("block1_conv1")
1

>>> model.layers[1].set_weights(weights)
# This will set the weights for that particular layer.

With a for loop we can set_weights for the entire network.
"""
for i in layer_dict.keys():
    weight_names = f["model_weights"][i].attrs["weight_names"]
    weights = [f["model_weights"][i][j] for j in weight_names]
    index = layer_names.index(i)
    model.layers[index].set_weights(weights)


# for layer in model.layers:
#     layer.trainable = False

#Adding custom Layers 
x = model.output
x = Conv2D(128, (3, 3), activation='relu', strides=(2, 2), padding='same', name='block3_conv1')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)
x = Conv2D(256, (3, 3), activation='relu', strides=(2, 2), padding='same', name='block4_conv1')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(32, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation="softmax")(x)

# creating the final model 
model_final = Model(inputs = model.input, outputs = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])
model_final.summary()

In [3]:
from glob import glob
import numpy as np
from PIL import Image


def load_image( infilename ) :
    img = Image.open( infilename )
    img.load()
    data = np.asarray( img, dtype="float32" )/255.0
    return data

good_x = np.asarray(list(map(load_image, glob('train/good/*.jpg'))))
good_y = np.asarray([1]*len(good_x), dtype="float32")
bad_x = np.asarray(list(map(load_image, glob('train/bad/*.jpg'))))
bad_y = np.asarray([0]*len(bad_x), dtype="float32")

# good_x[:,:,:,:] = 0.0
# good_x[:,:,:,1] = 0.5
# bad_x[:,:,:,:] = 0.0
# bad_x[:,:,:,0] = 0.5
x = np.concatenate((good_x, bad_x), axis=0)
y = np.concatenate((good_y, bad_y), axis=0)

val_good_x = np.asarray(list(map(load_image, glob('val/good/*.jpg'))))
val_good_y = np.asarray([1]*len(val_good_x), dtype="float32")
val_bad_x = np.asarray(list(map(load_image, glob('val/bad/*.jpg'))))
val_bad_y = np.asarray([0]*len(val_bad_x), dtype="float32")

# val_good_x[:,:,:,:] = 0.0
# val_good_x[:,:,:,1] = 0.5
# val_bad_x[:,:,:,:] = 0.0
# val_bad_x[:,:,:,0] = 0.5
val_x = np.concatenate((val_good_x, val_bad_x), axis=0)
val_y = np.concatenate((val_good_y, val_bad_y), axis=0)



In [17]:
import numpy as np
good_x_1 = np.full((10000, 224, 224, 3), 0, dtype='float32')
good_x_1[:,:,:,1] = 0.5
bad_x_1 = np.full((10000, 224, 224, 3), 0, dtype='float32')
bad_x_1[:,:,:,0] = 0.5
good_y_1 = np.asarray([1]*len(good_x_1), dtype="float32")
bad_y_1 = np.asarray([0]*len(bad_x_1), dtype="float32")

x_1 = np.concatenate((good_x_1, bad_x_1), axis=0)
y_1 = np.concatenate((good_y_1, bad_y_1), axis=0)

val_good_x_1 = np.full((2000, 224, 224, 3), 0, dtype='float32')
val_good_x_1[:,:,:,1] = 0.5
val_bad_x_1 = np.full((2000, 224, 224, 3), 0, dtype='float32')
val_bad_x_1[:,:,:,0] = 0.5
val_good_y_1 = np.asarray([1]*len(val_good_x_1), dtype="float32")
val_bad_y_1 = np.asarray([0]*len(val_bad_x_1), dtype="float32")

val_x_1 = np.concatenate((val_good_x_1, val_bad_x_1), axis=0)
val_y_1 = np.concatenate((val_good_y_1, val_bad_y_1), axis=0)

# s = np.arange(x_1.shape[0])
# np.random.shuffle(s)
# x_1 = x_1[s]
# y_1 = y_1[s]

In [4]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("densenet.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')
# plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2)

# Train the model 
model_final.fit(x=x,
          y=y,
          batch_size=batch_size,
          epochs=100,
#           validation_split=0.2,
          validation_data=(val_x, val_y),
          shuffle=True,
          callbacks = [checkpoint, early])

Train on 20000 samples, validate on 4000 samples
Epoch 1/100
20000/20000 [==============================] - 261s 13ms/step - loss: 0.0560 - acc: 0.9888 - val_loss: 0.2302 - val_acc: 1.0000

Epoch 00001: val_loss improved from inf to 0.23017, saving model to densenet.h5
Epoch 2/100
20000/20000 [==============================] - 256s 13ms/step - loss: 0.0290 - acc: 0.9961 - val_loss: 0.1949 - val_acc: 1.0000

Epoch 00002: val_loss improved from 0.23017 to 0.19488, saving model to densenet.h5
Epoch 3/100
20000/20000 [==============================] - 256s 13ms/step - loss: 0.0246 - acc: 0.9949 - val_loss: 0.0205 - val_acc: 1.0000

Epoch 00003: val_loss improved from 0.19488 to 0.02051, saving model to densenet.h5
Epoch 4/100
20000/20000 [==============================] - 256s 13ms/step - loss: 0.0230 - acc: 0.9945 - val_loss: 0.0079 - val_acc: 1.0000

Epoch 00004: val_loss improved from 0.02051 to 0.00788, saving model to densenet.h5
Epoch 5/100
20000/20000 [==============================

KeyboardInterrupt: 

In [27]:
model_final.evaluate(val_x_1,val_y_1, verbose=1) 

400/400 [==============================] - 4s 11ms/step


[0.49575607538223265, 0.5]